In [22]:
import os
import gzip
import cPickle
import wget
%matplotlib inline  

import numpy as np


def load_mnist():
    if not os.path.exists(os.path.join(os.curdir, 'data')):
        os.mkdir(os.path.join(os.curdir, 'data'))
        wget.download('http://deeplearning.net/data/mnist/mnist.pkl.gz', out='data')

    data_file = gzip.open(os.path.join(os.curdir, 'data', 'mnist.pkl.gz'), 'rb')
    training_data, validation_data, test_data = cPickle.load(data_file)
    data_file.close()

    training_inputs = [np.reshape(x, (784, 1)) for x in training_data[0]]
    training_results = [vectorized_result(y) for y in training_data[1]]
    training_data = zip(training_inputs, training_results)

    validation_inputs = [np.reshape(x, (784, 1)) for x in validation_data[0]]
    validation_results = validation_data[1]
    validation_data = zip(validation_inputs, validation_results)

    test_inputs = [np.reshape(x, (784, 1)) for x in test_data[0]]
    test_data = zip(test_inputs, test_data[1])
    return training_data, validation_data, test_data


def vectorized_result(y):
    e = np.zeros((10, 1))
    e[y] = 1.0
    return e

In [1]:
from PIL import Image

In [46]:
import skimage.measure
def pooling_data(data, basepixel, pooling_size):
    #print pd.DataFrame(data)

    #大于某个像素值 ==> 255
    pooled_data = skimage.measure.block_reduce(data,pooling_size,  np.mean)
    if basepixel > 0:
        pooled_data[pooled_data> basepixel] = 255
        pooled_data[pooled_data<= basepixel] = 0
    #压缩矩阵操作,取平均值，你可以自己写函数
    return pooled_data


In [47]:
import os


def save_to_image(pixels,basepixel,size, i):
    image = Image.new('L',pixels.shape)
    columns = pixels.shape[0]
    rows = pixels.shape[1]
    
    for x in xrange(columns):
            for y in xrange(rows):
                    pix = pixels[y][x]
                    image.putpixel((y, x), int(pixels[y][x]))
    
    
    directory = "pooling/"+"basepixel_"+str(basepixel) +"/"+ str(columns)+"_"+str(rows)
    print directory+str(i)
    #print pd.DataFrame(pixels)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    image.save(directory+"/"+str(i) + '.png')

In [53]:
def save_to_numpy(MINST_SAVE, basepixel, size):
    directory = "MINST_train/"+"basepixel_"+str(basepixel) +"/"+ str(size[0])+"_"+str(size[1])+"/"
    
    #print pd.DataFrame(pixels)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    np.save(directory+'MINST_train.npy', MINST_SAVE)
    test = np.load(directory+'MINST_train.npy')
    print np.all(MINST_SAVE == test)  
    
    print MINST_SAVE.shape
    with open(directory+'MINST_train.txt','wb') as f:
        for row in MINST_SAVE:
             np.savetxt(f,np.transpose(row),delimiter=',',fmt = "%03d", footer='\n')
    x = np.loadtxt(directory+'MINST_train.txt',delimiter=',')
    print np.all(x == MINST_SAVE)

In [54]:
int(5.6)

5

In [64]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


from PIL import Image
import struct
import numpy as np
import pandas as pd
import math

def deal_image(filename, pooling_size_list, Debug_num = 0):
    f = open(filename, 'rb')

    index = 0
    buf = f.read()

    print "buf", len(buf)
    f.close()
    

    magic, images, rows, columns = struct.unpack_from('>IIII' , buf , index)
    index += struct.calcsize('>IIII')
    if Debug_num == 0:
        images_num = images
    else:
        images_num = Debug_num
    
    for size in pooling_size_list:
        for basepixel in range(0, 256, 25):
            
            pooled_size = (int(math.ceil(columns*1.0/size[0])) , int(math.ceil(rows*1.0/size[1])))
            print pooled_size
            np_to_save = np.zeros((images_num,  pooled_size[0], pooled_size[1]))
                                  
            for i in xrange(images_num):
                # for debug
                

                normal_np = np.zeros((columns, rows))

                #冗余的代码
                for x in xrange(columns):
                  for y in xrange(rows):
                    normal_np[y][x] = int(struct.unpack_from('>B', buf, index)[0])
                    index += struct.calcsize('>B')



                    #print normal_np.shape
                print np_to_save[i].shape
                pooled_data = pooling_data(normal_np, basepixel, size)
                
                np_to_save[i] = pooled_data
                #print np_to_save[i]
        
                save_to_image(pooled_data,basepixel,size, i)
            
            save_to_numpy(np_to_save, basepixel, pooled_size)
        
        




In [65]:
#设置要池化的数据块大小
pooling_size_list = [(1,1), (2,2),(4,4),(5,5), (7,7)]

#if Debug_num == 0, then save all images, Debug_num >0, save Debug_num images
deal_image("train-images.idx3-ubyte", pooling_size_list,Debug_num=10)

buf 47040016
(28, 28)
(28, 28)
pooling/basepixel_0/28_280
(28, 28)
pooling/basepixel_0/28_281
(28, 28)
pooling/basepixel_0/28_282
(28, 28)
pooling/basepixel_0/28_283
(28, 28)
pooling/basepixel_0/28_284
(28, 28)
pooling/basepixel_0/28_285
(28, 28)
pooling/basepixel_0/28_286
(28, 28)
pooling/basepixel_0/28_287
(28, 28)
pooling/basepixel_0/28_288
(28, 28)
pooling/basepixel_0/28_289
True
(10, 28, 28)
False
(28, 28)
(28, 28)
pooling/basepixel_25/28_280
(28, 28)
pooling/basepixel_25/28_281
(28, 28)
pooling/basepixel_25/28_282
(28, 28)
pooling/basepixel_25/28_283
(28, 28)
pooling/basepixel_25/28_284
(28, 28)
pooling/basepixel_25/28_285
(28, 28)
pooling/basepixel_25/28_286
(28, 28)
pooling/basepixel_25/28_287
(28, 28)
pooling/basepixel_25/28_288
(28, 28)
pooling/basepixel_25/28_289
True
(10, 28, 28)
False
(28, 28)
(28, 28)
pooling/basepixel_50/28_280
(28, 28)
pooling/basepixel_50/28_281
(28, 28)
pooling/basepixel_50/28_282
(28, 28)
pooling/basepixel_50/28_283
(28, 28)
pooling/basepixel_50/28_2

/home/daneel/anaconda3/envs/myquant/lib/python2.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


False
(28, 28)
(28, 28)
pooling/basepixel_75/28_280
(28, 28)
pooling/basepixel_75/28_281
(28, 28)
pooling/basepixel_75/28_282
(28, 28)
pooling/basepixel_75/28_283
(28, 28)
pooling/basepixel_75/28_284
(28, 28)
pooling/basepixel_75/28_285
(28, 28)
pooling/basepixel_75/28_286
(28, 28)
pooling/basepixel_75/28_287
(28, 28)
pooling/basepixel_75/28_288
(28, 28)
pooling/basepixel_75/28_289
True
(10, 28, 28)
False
(28, 28)
(28, 28)
pooling/basepixel_100/28_280
(28, 28)
pooling/basepixel_100/28_281
(28, 28)
pooling/basepixel_100/28_282
(28, 28)
pooling/basepixel_100/28_283
(28, 28)
pooling/basepixel_100/28_284
(28, 28)
pooling/basepixel_100/28_285
(28, 28)
pooling/basepixel_100/28_286
(28, 28)
pooling/basepixel_100/28_287
(28, 28)
pooling/basepixel_100/28_288
(28, 28)
pooling/basepixel_100/28_289
True
(10, 28, 28)
False
(28, 28)
(28, 28)
pooling/basepixel_125/28_280
(28, 28)
pooling/basepixel_125/28_281
(28, 28)
pooling/basepixel_125/28_282
(28, 28)
pooling/basepixel_125/28_283
(28, 28)
pooling/

(7, 7)
pooling/basepixel_100/7_71
(7, 7)
pooling/basepixel_100/7_72
(7, 7)
pooling/basepixel_100/7_73
(7, 7)
pooling/basepixel_100/7_74
(7, 7)
pooling/basepixel_100/7_75
(7, 7)
pooling/basepixel_100/7_76
(7, 7)
pooling/basepixel_100/7_77
(7, 7)
pooling/basepixel_100/7_78
(7, 7)
pooling/basepixel_100/7_79
True
(10, 7, 7)
False
(7, 7)
(7, 7)
pooling/basepixel_125/7_70
(7, 7)
pooling/basepixel_125/7_71
(7, 7)
pooling/basepixel_125/7_72
(7, 7)
pooling/basepixel_125/7_73
(7, 7)
pooling/basepixel_125/7_74
(7, 7)
pooling/basepixel_125/7_75
(7, 7)
pooling/basepixel_125/7_76
(7, 7)
pooling/basepixel_125/7_77
(7, 7)
pooling/basepixel_125/7_78
(7, 7)
pooling/basepixel_125/7_79
True
(10, 7, 7)
False
(7, 7)
(7, 7)
pooling/basepixel_150/7_70
(7, 7)
pooling/basepixel_150/7_71
(7, 7)
pooling/basepixel_150/7_72
(7, 7)
pooling/basepixel_150/7_73
(7, 7)
pooling/basepixel_150/7_74
(7, 7)
pooling/basepixel_150/7_75
(7, 7)
pooling/basepixel_150/7_76
(7, 7)
pooling/basepixel_150/7_77
(7, 7)
pooling/basepixel

True
(10, 4, 4)
False


In [26]:
def read_to_numpy(basepixel, size):
    directory = "MINST_train/"+"basepixel_"+str(basepixel) +"/"+ str(size[0])+"_"+str(size[1])+"/"
    
    #print pd.DataFrame(pixels)
    print directory
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
    test = np.load(directory+'MINST_train.npy')
    
    img = Image.fromarray(test[1], 'L')
    img.show()
    print pd.DataFrame(test[1])

In [27]:
read_to_numpy(125, (28,28))

MINST_train/basepixel_125/28_28/
     0    1    2    3    4      5      6      7      8      9  ...      18  \
0   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...     0.0   
1   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...     0.0   
2   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...     0.0   
3   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...     0.0   
4   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...     0.0   
5   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...   255.0   
6   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...   255.0   
7   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...   255.0   
8   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0    0.0    0.0 ...   255.0   
9   0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0  255.0  255.0 ...     0.0   
10  0.0  0.0  0.0  0.0  0.0    0.0    0.0    0.0  255.0  255.0 ...     0.0   
11  0.0  0.0  0.0  0.0  0.0    

In [19]:
import numpy as np
from PIL import Image

# Creates a random image 100*100 pixels
mat = np.random.random((100,100))

# Creates PIL image
img = Image.fromarray(mat, 'L')
img.show()

In [44]:
28/3

9

In [44]:
print np.zeros((2,3,2))

[[[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]]
